In [ ]:
!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
pip install numpy==1.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


In [ ]:
import os
from glob import glob
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, Resized, RandRotate90d, RandFlipd, ToTensord
from monai.data import Dataset, DataLoader
from monai.networks.nets import DenseNet121

In [ ]:
from google.colab import drive
import os

# Google Drive'ı mount et
drive.mount('/content/drive')

# Veri dizininiz (içinde "0" ve "1" klasörleri bulunuyor)
data_dir = "/content/drive/MyDrive/BeyinInmeVerisi-Siniflandirma"

Mounted at /content/drive


In [ ]:
# Dosya yolları ve etiket listesini oluşturma
data_files = []
for label in ['0', '1']:
    image_paths = glob(os.path.join(data_dir, label, "*"))
    for path in image_paths:
        data_files.append({"image": path, "label": int(label)})

print(f"Toplam örnek sayısı: {len(data_files)}")

Toplam örnek sayısı: 6947


In [ ]:
# Eğitim ve doğrulama setine ayırma (%80 - %20, stratified)
train_files, temp_files = train_test_split(
    data_files, test_size=0.2, random_state=42,
    stratify=[d["label"] for d in data_files]
)

# Doğrulama ve test setine ayırma (%50 - %50, stratified)
val_files, test_files = train_test_split(
    temp_files, test_size=0.5, random_state=42,
    stratify=[d["label"] for d in temp_files]
)

# MONAI transformları tanımlama
train_transforms = Compose([
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"]),
    ScaleIntensityd(keys=["image"]),
    Resized(keys=["image"], spatial_size=(224, 224)),
    RandRotate90d(keys=["image"], prob=0.5, spatial_axes=(0, 1)),
    RandFlipd(keys=["image"], prob=0.5, spatial_axis=0),
    ToTensord(keys=["image"]),
])

val_transforms = Compose([
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"]),
    ScaleIntensityd(keys=["image"]),
    Resized(keys=["image"], spatial_size=(224, 224)),
    ToTensord(keys=["image"]),
])

test_transforms = Compose([
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"]),
    ScaleIntensityd(keys=["image"]),
    Resized(keys=["image"], spatial_size=(224, 224)),
    ToTensord(keys=["image"]),
])

batch_size = 32

# Test dataset ve DataLoader oluştur
test_dataset = Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# MONAI ImageDataset ile veri setlerini oluşturma
train_dataset = Dataset(data=train_files, transform=train_transforms)
val_dataset = Dataset(data=val_files, transform=val_transforms)

# DataLoader ayarları
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Cihaz ayarı (GPU varsa kullanılıyor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Cihaz:", device)


# MONAI DenseNet121 modelini oluşturma
# - spatial_dims=2: 2D görüntüler için
# - in_channels=3: Renkli görüntüler (eğer veriniz grayscale ise 1 olarak ayarlayın)
# - out_channels=2: Binary sınıflandırma (0 ve 1)
model = DenseNet121(spatial_dims=2, in_channels=3, out_channels=2)
model = model.to(device)

# Loss fonksiyonu ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Cihaz: cuda


In [ ]:
sample_batch = next(iter(train_loader))
print(sample_batch["image"].shape)  # (batch_size, channels, height, width)


torch.Size([32, 4, 224, 224])


In [ ]:
# Eğitim döngüsü
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in train_loader:
        images = batch["image"][:, :3, :, :].to(device)  # İlk 3 kanalı al
        labels = batch["label"].to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}")

    # Doğrulama aşaması
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for batch in val_loader:
            images = batch["image"][:, :3, :, :].to(device)  # İlk 3 kanalı al
            labels = batch["label"].to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
    val_epoch_loss = val_loss / total_val
    val_epoch_acc = correct_val / total_val
    print(f"Validation - Loss: {val_epoch_loss:.4f} - Accuracy: {val_epoch_acc:.4f}")


torch.save(model, "monai_beyin_inme_model_full.pth")
torch.save(model, "/content/drive/MyDrive/monai_beyin_inme_model_full.pth")


Epoch 1/10 - Loss: 0.4577 - Accuracy: 0.7869
Validation - Loss: 0.3374 - Accuracy: 0.8518
Epoch 2/10 - Loss: 0.3439 - Accuracy: 0.8535
Validation - Loss: 0.2867 - Accuracy: 0.8921
Epoch 3/10 - Loss: 0.3140 - Accuracy: 0.8688
Validation - Loss: 0.3501 - Accuracy: 0.8388
Epoch 4/10 - Loss: 0.2903 - Accuracy: 0.8789
Validation - Loss: 0.2716 - Accuracy: 0.9007
Epoch 5/10 - Loss: 0.2588 - Accuracy: 0.9005
Validation - Loss: 0.2898 - Accuracy: 0.8835
Epoch 6/10 - Loss: 0.2531 - Accuracy: 0.8981
Validation - Loss: 0.2297 - Accuracy: 0.9137
Epoch 7/10 - Loss: 0.2273 - Accuracy: 0.9061
Validation - Loss: 0.3226 - Accuracy: 0.8791
Epoch 8/10 - Loss: 0.2053 - Accuracy: 0.9208
Validation - Loss: 0.2998 - Accuracy: 0.8705
Epoch 9/10 - Loss: 0.2083 - Accuracy: 0.9194
Validation - Loss: 0.2533 - Accuracy: 0.9007
Epoch 10/10 - Loss: 0.1885 - Accuracy: 0.9286
Validation - Loss: 0.1731 - Accuracy: 0.9439


In [ ]:
# Modeli test moduna al
model.eval()

test_loss = 0.0
correct_test = 0
total_test = 0

with torch.no_grad():
    for batch in test_loader:
        images = batch["image"][:, :3, :, :].to(device)  # İlk 3 kanalı al
        labels = batch["label"].to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct_test += (preds == labels).sum().item()
        total_test += labels.size(0)

test_epoch_loss = test_loss / total_test
test_epoch_acc = correct_test / total_test

print(f"Test Sonuçları - Loss: {test_epoch_loss:.4f} - Accuracy: {test_epoch_acc:.4f}")


Test Sonuçları - Loss: 0.1781 - Accuracy: 0.9338
